In [6]:
import sys

sys.path.append('../')

In [11]:
# import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd

import os
from glob import glob, escape
import torch.nn as nn
from torch.nn import functional as F

import pickle
import yaml
from easydict import EasyDict as edict

In [12]:
from models.model import DimeNet

In [13]:
from dataset.dataset_loader import get_dataset

In [14]:
config_file = glob('../config/config.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [15]:
dimenet = DimeNet(
            config.model
        )

In [16]:
train, valid, test = get_dataset('../data')

In [17]:
from torch_geometric.loader import DataLoader

In [39]:
loader = DataLoader(test, batch_size=1)

In [40]:
inpt = next(iter(loader))

In [41]:
inpt

DataBatch(x=[26, 18], edge_index=[2, 60], edge_attr=[60, 4], y=[1], pos=[26, 3], z=[26], idx=[1], state=[1], batch=[26], ptr=[2])

In [42]:
out = dimenet(z=inpt.z, pos=inpt.pos, batch=inpt.batch)

In [43]:
submission = pd.read_csv("../data/sample_submission.csv", index_col=0)

In [59]:
out.cpu().detach().numpy() + 1

array([[1.]], dtype=float32)

In [72]:
if inpt.state[0] == 'g':
    submission.iloc[int(inpt.idx[0])]['Reorg_g'] = out.cpu().detach().numpy() + 0.01
else:
    submission.iloc[int(inpt.idx[0])]['Reorg_ex'] = out.cpu().detach().numpy()

In [73]:
submission.iloc[182]

Reorg_g     0.01
Reorg_ex    0.00
Name: test_182, dtype: float64

In [67]:
submission.dtypes

Reorg_g     int64
Reorg_ex    int64
dtype: object

In [71]:
submission = submission.astype('float')

In [63]:
submission.to_csv('./submission.csv')